In [15]:
import pandas as pd
from validator_collection import checkers
import re
import matplotlib.pyplot as plt
import collections

In [16]:
df = pd.read_csv('../data/tweets_10-24-2020.csv')
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54442 entries, 0 to 54441
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   id         54442 non-null  int64 
 1   text       54442 non-null  object
 2   isRetweet  54442 non-null  object
 3   isDeleted  54442 non-null  object
 4   device     54442 non-null  object
 5   favorites  54442 non-null  int64 
 6   retweets   54442 non-null  int64 
 7   date       54442 non-null  object
dtypes: int64(3), object(5)
memory usage: 3.3+ MB


In [17]:
# Convert to date time
df['date']= pd.to_datetime(df['date'])
df.info()
df

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 54442 entries, 0 to 54441
Data columns (total 8 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   id         54442 non-null  int64         
 1   text       54442 non-null  object        
 2   isRetweet  54442 non-null  object        
 3   isDeleted  54442 non-null  object        
 4   device     54442 non-null  object        
 5   favorites  54442 non-null  int64         
 6   retweets   54442 non-null  int64         
 7   date       54442 non-null  datetime64[ns]
dtypes: datetime64[ns](1), int64(3), object(4)
memory usage: 3.3+ MB


,id,text,isRetweet,isDeleted,device,favorites,retweets,date
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50
2,1218010753434820614,RT @CBS_Herridge: READ: Letter to surveillance...,t,f,Twitter for iPhone,0,7396,2020-01-17 03:22:47
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58
4,1218159531554897920,RT @MZHemingway: Very friendly telling of even...,t,f,Twitter for iPhone,0,9081,2020-01-17 13:13:59
...,...,...,...,...,...,...,...,...
54437,1319484210101379072,RT @EliseStefanik: President @realDonaldTrump ...,t,f,Twitter for iPhone,0,9912,2020-10-23 03:42:05
54438,1319444420861829121,RT @TeamTrump: LIVE: Presidential Debate #Deba...,t,f,Twitter for iPhone,0,8249,2020-10-23 01:03:58
54439,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,175950,36098,2020-10-22 21:04:21
54440,1319345719829008387,Suburban women want Safety &amp; Security. Joe...,f,f,Twitter for iPhone,95325,19639,2020-10-22 18:31:46


In [18]:
# Filtering
df = df.loc[df['isRetweet'] == 'f']
df



,id,text,isRetweet,isDeleted,device,favorites,retweets,date
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58
5,1319761576996573186,THANK YOU to all of the Great American Patriot...,f,f,Twitter for iPhone,100564,18992,2020-10-23 22:04:14
7,1315779944002199552,"“I’m running as a proud Democrat, for the Sena...",f,f,Twitter for iPhone,149007,34897,2020-10-12 22:22:39
...,...,...,...,...,...,...,...,...
54429,1212177432452698115,Thank you to the @dcexaminer Washington Examin...,f,f,Twitter for iPhone,35044,9213,2020-01-01 01:03:15
54430,1212175360093229056,One of my greatest honors was to have gotten C...,f,f,Twitter for iPhone,56731,12761,2020-01-01 00:55:01
54439,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,175950,36098,2020-10-22 21:04:21
54440,1319345719829008387,Suburban women want Safety &amp; Security. Joe...,f,f,Twitter for iPhone,95325,19639,2020-10-22 18:31:46


In [19]:
def contains_url(text):
    return checkers.is_url(text)

def strip_url(text):
    return re.sub(r'^https?:\/\/.*[\r\n]*', '', text, flags=re.MULTILINE)

def remove_URL(sample):
    """Remove URLs from a sample string"""
    return re.sub(r"http\S+", "", sample)

In [20]:
# Label the records where it is a url with a new column 'is_url' with boolean
df['is_url'] = df.apply(lambda x: contains_url(x['text']),axis=1)
df['text'] = df.apply(lambda x: remove_URL(x['text']),axis=1)
df

<ipython-input-20-525629103b06>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['is_url'] = df.apply(lambda x: contains_url(x['text']),axis=1)
<ipython-input-20-525629103b06>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['text'] = df.apply(lambda x: remove_URL(x['text']),axis=1)


,id,text,isRetweet,isDeleted,device,favorites,retweets,date,is_url
0,98454970654916608,Republicans and Democrats have both created ou...,f,f,TweetDeck,49,255,2011-08-02 18:07:48,False
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,False
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,False
5,1319761576996573186,THANK YOU to all of the Great American Patriot...,f,f,Twitter for iPhone,100564,18992,2020-10-23 22:04:14,False
7,1315779944002199552,"“I’m running as a proud Democrat, for the Sena...",f,f,Twitter for iPhone,149007,34897,2020-10-12 22:22:39,False
...,...,...,...,...,...,...,...,...,...
54429,1212177432452698115,Thank you to the @dcexaminer Washington Examin...,f,f,Twitter for iPhone,35044,9213,2020-01-01 01:03:15,False
54430,1212175360093229056,One of my greatest honors was to have gotten C...,f,f,Twitter for iPhone,56731,12761,2020-01-01 00:55:01,False
54439,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,175950,36098,2020-10-22 21:04:21,False
54440,1319345719829008387,Suburban women want Safety &amp; Security. Joe...,f,f,Twitter for iPhone,95325,19639,2020-10-22 18:31:46,False


In [21]:
# Show only the records where 'is_url' is True
# df[df['is_url'] == True]
mask = (df['is_url'] == False) & (df['date'] > '2016-01-01')
filtered_dates = df.loc[mask]

filtered_dates.to_csv('../data/filtered_tweets.csv')
filtered_dates
# df[df['is_url'] == False & df['date'] > '2016-01-01']

,id,text,isRetweet,isDeleted,device,favorites,retweets,date,is_url
1,1234653427789070336,I was thrilled to be back in the Great city of...,f,f,Twitter for iPhone,73748,17404,2020-03-03 01:34:50,False
3,1304875170860015617,The Unsolicited Mail In Ballot Scam is a major...,f,f,Twitter for iPhone,80527,23502,2020-09-12 20:10:58,False
5,1319761576996573186,THANK YOU to all of the Great American Patriot...,f,f,Twitter for iPhone,100564,18992,2020-10-23 22:04:14,False
7,1315779944002199552,"“I’m running as a proud Democrat, for the Sena...",f,f,Twitter for iPhone,149007,34897,2020-10-12 22:22:39,False
8,1223640662689689602,Getting a little exercise this morning!,f,f,Twitter for iPhone,285863,30209,2020-02-01 16:14:02,False
...,...,...,...,...,...,...,...,...,...
54429,1212177432452698115,Thank you to the @dcexaminer Washington Examin...,f,f,Twitter for iPhone,35044,9213,2020-01-01 01:03:15,False
54430,1212175360093229056,One of my greatest honors was to have gotten C...,f,f,Twitter for iPhone,56731,12761,2020-01-01 00:55:01,False
54439,1319384118849949702,Just signed an order to support the workers of...,f,f,Twitter for iPhone,175950,36098,2020-10-22 21:04:21,False
54440,1319345719829008387,Suburban women want Safety &amp; Security. Joe...,f,f,Twitter for iPhone,95325,19639,2020-10-22 18:31:46,False
